<a href="https://colab.research.google.com/github/hifistelija/ML_Model_V3.1_Ennustukset/blob/main/ML_Model_V3_1_ennustukset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kirjastojen tuonti

In [ ]:
import pandas as pd
import numpy as np
import os
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Mallin sisäänluku

In [ ]:
# os.chdir("/content/drive/MyDrive/MasterData/ML_Model_V3/Final_Model")

In [ ]:
path = "/content/drive/MyDrive/MasterData/ML_Model_V3/Final_Model/"

In [ ]:
with open(path + 'ML_Model_V3.pkl', 'rb') as f:
  loaded_model = pickle.load(f)

In [ ]:
print(type(loaded_model))

<class 'sklearn.ensemble._forest.RandomForestRegressor'>


In [ ]:
with open(path + 'ML_Model_V3_Columns.pkl', 'rb') as f:
  loaded_columns = pickle.load(f)

In [ ]:
print(type(loaded_columns))

<class 'list'>


In [ ]:
with open(path + 'label_encoders.pkl', 'rb') as f:
  loaded_encoders = pickle.load(f)

loaded_day_encoder = loaded_encoders['encoder_day']
loaded_item_encoder = loaded_encoders['encoder_item']

In [ ]:
print(type(loaded_day_encoder), type(loaded_item_encoder))

<class 'sklearn.preprocessing._label.LabelEncoder'> <class 'sklearn.preprocessing._label.LabelEncoder'>


# Uuden datan tuonti ennustetta varten

In [ ]:
# os.chdir('/content')

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ML_uusi_data.xlsx to ML_uusi_data.xlsx


In [ ]:
new_data = pd.read_excel("ML_uusi_data.xlsx")

In [ ]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49255 entries, 0 to 49254
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Linjakoodi         49255 non-null  int64  
 1   Päivän nimi        49255 non-null  object 
 2   Ennuste            48240 non-null  float64
 3   Nimikkeen nro      49255 non-null  object 
 4   Kuvaus             49255 non-null  object 
 5   Perusmittayksikkö  49255 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 2.3+ MB


In [ ]:
new_data_copy = new_data # kopio dataframesta poistettujen sarakkeiden uudelleen tuontia varten

# Uuden datan muokkaukset

In [ ]:
new_data = new_data.dropna()

In [ ]:
new_data.rename(columns={"Ennuste": "Varaukset"}, inplace=True)

<ipython-input-15-00b4c3c0014e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data.rename(columns={"Ennuste": "Varaukset"}, inplace=True)


In [ ]:
new_data["Päivän nimi"] = loaded_day_encoder.transform(new_data["Päivän nimi"])
new_data["Nimikkeen nro"] = loaded_item_encoder.transform(new_data["Nimikkeen nro"])

<ipython-input-16-094082904b37>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data["Päivän nimi"] = loaded_day_encoder.transform(new_data["Päivän nimi"])
<ipython-input-16-094082904b37>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data["Nimikkeen nro"] = loaded_item_encoder.transform(new_data["Nimikkeen nro"])


In [ ]:
columns_to_drop = ["Perusmittayksikkö", "Kuvaus"]  # list of columns to drop

In [ ]:
for col in columns_to_drop:
  if col in new_data.columns:
    new_data.drop(col, axis=1, inplace=True)

<ipython-input-18-46c3b6d70bb9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data.drop(col, axis=1, inplace=True)
<ipython-input-18-46c3b6d70bb9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data.drop(col, axis=1, inplace=True)


In [ ]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48240 entries, 0 to 49253
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Linjakoodi     48240 non-null  int64  
 1   Päivän nimi    48240 non-null  int64  
 2   Varaukset      48240 non-null  float64
 3   Nimikkeen nro  48240 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 1.8 MB


# Menekki ennuste

In [ ]:
new_data = new_data[loaded_columns]

In [ ]:
new_data.head()

,Päivän nimi,Linjakoodi,Varaukset,Nimikkeen nro
0,4,73,370.0,235
1,4,73,370.0,70
2,4,73,370.0,60
3,4,73,370.0,245
4,4,73,370.0,173


In [ ]:
predictions = loaded_model.predict(new_data)

In [ ]:
new_data["Määrä"] = predictions

In [ ]:
new_data.head(10)

,Päivän nimi,Linjakoodi,Varaukset,Nimikkeen nro,Määrä
0,4,73,370.0,235,1.107211
1,4,73,370.0,70,2.003403
2,4,73,370.0,60,2.891139
3,4,73,370.0,245,4.003262
4,4,73,370.0,173,4.312841
5,4,73,370.0,161,11.522850
6,4,73,370.0,126,2.424461
7,4,73,370.0,121,1.078906
8,4,73,370.0,110,2.701544
9,4,73,370.0,104,6.815576


In [ ]:
new_data["Päivän nimi"] = loaded_day_encoder.inverse_transform(new_data["Päivän nimi"])
new_data["Nimikkeen nro"] = loaded_item_encoder.inverse_transform(new_data["Nimikkeen nro"])

In [ ]:
new_data.head(10)

,Päivän nimi,Linjakoodi,Varaukset,Nimikkeen nro,Määrä
0,Sunnuntai,73,370.0,R01877,1.107211
1,Sunnuntai,73,370.0,30309,2.003403
2,Sunnuntai,73,370.0,22072,2.891139
3,Sunnuntai,73,370.0,R02038,4.003262
4,Sunnuntai,73,370.0,R00481,4.312841
5,Sunnuntai,73,370.0,R00025,11.522850
6,Sunnuntai,73,370.0,70177,2.424461
7,Sunnuntai,73,370.0,70088,1.078906
8,Sunnuntai,73,370.0,50302,2.701544
9,Sunnuntai,73,370.0,50134,6.815576


In [ ]:
new_data.to_excel("ML_myynti_ennuste.xlsx", index=False)